In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
import plotly.graph_objects as go
from skimage import io
import plotly.express as px
import time

In [2]:
Saturday = pd.DataFrame(pd.read_csv('park-movement-Sat_NewII.csv'))
Saturday = Saturday.drop(["Unnamed: 0"], axis=1)
Saturday["id"] = Saturday["id"].astype(str)

In [3]:
Saturday

,Timestamp,id,type,X,Y,visit_count,visit_duration,Facility_coordinates
0,1402149608,1102394,1,99,77,1472,883,"(99, 77)"
1,1402149648,1102394,0,98,77,1472,883,"(98, 77)"
2,1402149655,1102394,0,97,77,1472,883,"(97, 77)"
3,1402149662,1102394,0,96,77,1472,883,"(96, 77)"
4,1402149669,1102394,0,95,76,1472,883,"(95, 76)"
...,...,...,...,...,...,...,...,...
9078618,1402204680,413545,0,4,67,737,323,"(4, 67)"
9078619,1402204685,413545,0,3,66,737,323,"(3, 66)"
9078620,1402204690,413545,0,2,67,737,323,"(2, 67)"
9078621,1402204695,413545,0,1,66,737,323,"(1, 66)"


In [4]:
d1 = pd.read_csv('SaturdayGroupCount.csv')
d2 = pd.read_csv('SaturdayCrowdCount.csv')
d3 = pd.read_csv('SaturdaySingleCount.csv')

Group = [  i.split(".", 1)[0] for i in d1.columns ]
Crowd = [  i.split(".", 1)[0] for i in d2.columns ]
Single = [  i.split(".", 1)[0] for i in d3.columns ]

Saturday_Group = list(set(Group))
Saturday_Crowd = list(set(Crowd)-set(Group))
Saturday_Single=  list(set(list(set(Single)-set(Group)))-set(Crowd)) 

In [5]:
TimePointMorning = time.mktime(time.strptime("2014-6-07 11:00:00", "%Y-%m-%d %H:%M:%S"))
TimePointAfterNoon = time.mktime(time.strptime("2014-6-07 16:00:00", "%Y-%m-%d %H:%M:%S"))

ArrivalTime = Saturday.copy().drop_duplicates(subset='id')
ArrivalTime["ArrivalTime"] = ArrivalTime["Timestamp"].map(lambda x: 1 if x- TimePointMorning >= 0 and x - TimePointAfterNoon <= 0 else ( 2 if x - TimePointAfterNoon >= 0 else 0 ) )
Early = list(ArrivalTime[ArrivalTime["ArrivalTime"]==0].id.unique())
Noon =  list(ArrivalTime[ArrivalTime["ArrivalTime"]==1].id.unique())
Late = list(ArrivalTime[ArrivalTime["ArrivalTime"]==2].id.unique())
len(Saturday.id.unique()), len(Early), len(Noon), len(Late)

(6411, 6246, 116, 49)

In [6]:
np.mean(Saturday["visit_duration"].unique())
HighDuration = list(Saturday[Saturday["visit_duration"] >= np.mean(Saturday["visit_duration"].unique())].id.unique())
LowDuration = list(Saturday[Saturday["visit_duration"] < np.mean(Saturday["visit_duration"].unique())].id.unique())
len(Saturday.id.unique()), len(HighDuration), len(LowDuration)

(6411, 5933, 478)

In [33]:
np.mean(Saturday["visit_duration"].unique())


629.9705882352941

### Group 1

In [7]:
G1 = list(set(Saturday_Group)&set(Early)&set( HighDuration))
G1P = Saturday.loc[Saturday['id'].isin(G1)]
len(G1), round(len(G1)/len(Saturday.id.unique()),3)

(4400, 0.686)

In [8]:
GroupV = pd.DataFrame(G1P[G1P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
8,27,15,4468
17,47,11,4329
36,86,44,4270
16,45,24,4136
12,38,90,4109


### Group 2

In [9]:
G2 = list(set(Saturday_Crowd)&set(Early)&set( HighDuration))
G2P = Saturday.loc[Saturday['id'].isin(G2)]
len(G2), round(len(G2)/len(Saturday.id.unique()),3)

(986, 0.154)

In [10]:
GroupV = pd.DataFrame(G2P[G2P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
8,27,15,1181
16,45,24,1088
12,38,90,1083
17,47,11,1081
36,86,44,1054


### Group 3

In [11]:
G3 = list(set(Saturday_Single)&set(Early)&set(HighDuration))
G3P = Saturday.loc[Saturday['id'].isin(G3)]
len(G3), round(len(G3)/len(Saturday.id.unique()),3)

(547, 0.085)

In [12]:
GroupV = pd.DataFrame(G3P[G3P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
16,45,24,716
8,27,15,694
12,38,90,679
17,47,11,635
36,86,44,617


### Group 4

In [13]:
G4 = list(set(Saturday_Group)&set(Noon))
G4P = Saturday.loc[Saturday['id'].isin(G4)]
len(G4), round(len(G4)/len(Saturday.id.unique()),3) 

(99, 0.015)

In [14]:
GroupV = pd.DataFrame(G4P[G4P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
9,32,33,110
19,63,99,95
15,45,24,83
16,47,11,73
2,16,66,69


### Group 5

In [15]:
G5 = list(set(Saturday_Crowd)&set(Noon))
G5P = Saturday.loc[Saturday['id'].isin(G5)]
len(G5) , round(len(G5)/len(Saturday.id.unique()),3)

(17, 0.003)

In [16]:
GroupV = pd.DataFrame(G5P[G5P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
9,32,33,17
19,63,99,15
26,78,48,13
16,47,11,11
2,16,66,10


### Group 6

In [17]:
G6 = list(set(Saturday_Single)&set(LowDuration))
G6P = Saturday.loc[Saturday['id'].isin(G6)]
len(G6) , round(len(G6)/len(Saturday.id.unique()),3)

(89, 0.014)

In [18]:
GroupV = pd.DataFrame(G6P[G6P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
31,86,44,100
17,47,11,97
16,45,24,92
8,27,15,81
23,69,44,79


### Group 7

In [23]:
G7 = list(set(Saturday_Group)&set(LowDuration)&set(Early))
G7P = Saturday.loc[Saturday['id'].isin(G7)]
len(G7), round(len(G7)/len(Saturday.id.unique()),3)

(203, 0.032)

In [24]:
GroupV = pd.DataFrame(G7P[G7P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
20,63,99,150
35,86,44,110
4,17,43,103
12,38,90,93
17,47,11,92


### Group 8

In [21]:
G8 = list(set(Saturday_Group)&set(Late))
G8P = Saturday.loc[Saturday['id'].isin(G8)]
len(G8), round(len(G8)/len(Saturday.id.unique()),3)

(47, 0.007)

In [22]:
GroupV = pd.DataFrame(G8P[G8P.type == 1].groupby(["X","Y"])["id"].count())
GroupV = GroupV.reset_index()
GroupV.columns = ["X","Y","count"]
GroupV.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
21,69,44,36
3,17,43,28
35,87,63,26
9,32,33,26
11,38,90,24


In [25]:
g1 = pd.read_csv('SaturdayGapID.csv')
g2 = pd.read_csv('SaturdayGapMax.csv')
g3 = pd.read_csv('SaturdayGapMed.csv')

GapID = [  i.split(".", 1)[0] for i in g1.columns ]
GapMax = [  i.split(".", 1)[0] for i in g2.columns ]
GapMed = [  i.split(".", 1)[0] for i in g3.columns ]

In [26]:
inds1 =  np.argsort(GapMax)
GapH = np.array(GapID)[inds1[-150:]]
GapHV = Saturday.loc[Saturday['id'].isin(GapH)]

GapHV2 = pd.DataFrame(GapHV[GapHV.type == 1].groupby(["X","Y"])["id"].count())
GapHV2 = GapHV2.reset_index()
GapHV2.columns = ["X","Y","count"]

In [27]:
GapHV2.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
22,69,44,234
8,27,15,175
28,78,48,141
4,17,43,128
12,38,90,128


In [28]:
inds2 =  np.argsort(GapMed) 
Faster = np.array(GapID)[inds2[0:150]]
FasterV = Saturday.loc[Saturday['id'].isin(Faster)]

FasterV2 = pd.DataFrame(FasterV[FasterV.type == 1].groupby(["X","Y"])["id"].count())
FasterV2 = FasterV2.reset_index()
FasterV2.columns = ["X","Y","count"]

In [29]:
FasterV2.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
17,47,11,211
16,45,24,193
12,38,90,183
36,86,44,166
8,27,15,155


In [30]:
inds3 =  np.argsort(GapMed)
Slower = np.array(GapID)[inds3[-150:]]
SlowerV = Saturday.loc[Saturday['id'].isin(Slower)]

SlowerV2 = pd.DataFrame(SlowerV[SlowerV.type == 1].groupby(["X","Y"])["id"].count())
SlowerV2 = SlowerV2.reset_index()
SlowerV2.columns = ["X","Y","count"]

In [31]:
SlowerV2.sort_values(by=['count'],ascending=False)[0:5]

,X,Y,count
36,86,44,194
8,27,15,192
3,16,66,173
17,47,11,169
16,45,24,156
